# Deep Learning Final Project

## Reference
- [Dataset Link](https://www.kaggle.com/datasets/hadiepratamatulili/anime-vs-cartoon-vs-human)


## Problem Statement

- Given a dataset contains images `Anime`, `Human`, `Cartoon`, and we are asked to classify into those three categoriesa
- It's a `Classification` Problem.
- Given Dataset is `Images`

## Solution
- By using `Pytorch's Neural Networks`

### Step - 1 Prepare Notebook

For download dataset from kaggle, we can use opendatasets library and for 

In [ ]:
!pip install opendatasets jovian --quiet --upgrade

In [ ]:
import os
import numpy as np
import opendatasets as od

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

### Step - 2 Download Dataset

In [ ]:
DATASET_URL = 'https://www.kaggle.com/datasets/hadiepratamatulili/anime-vs-cartoon-vs-human'
od.download(DATASET_URL)

Let's take and look at the parent directory which we had download

In [ ]:
PARENT_DIR = '/content/anime-vs-cartoon-vs-human/Data'

os.listdir(PARENT_DIR)

We have images seperated into folders. Let's find the sizes of each folders

In [ ]:
for i in os.listdir(PARENT_DIR):
    print(f"Folder {i} with Size {len(os.listdir(PARENT_DIR + '/' + i))}")

### Step - 3 Explore Dataset

Let's look again and explore our dataset

In [ ]:
from PIL import Image
import random

def plotRawImage(DATA_DIR, num):
    """
    Function to plot raw images from the directory
    """
    size = len(DATA_DIR)
    data = os.listdir(DATA_DIR)
    img = Image.open(DATA_DIR + '/' + data[num])
    print(f'Image Size: {img.size}')
    return img

In [ ]:
plotRawImage('/content/anime-vs-cartoon-vs-human/Data/anime', 5)

In [ ]:
plotRawImage('/content/anime-vs-cartoon-vs-human/Data/cartoon', 556)

On Exploring, we came to know that each image has it's `own size`. We can re-size all images in specific size, which makes computation easy

### Step - 4 Load and Preprocess Dataset

We can use various `pytorch's` class like `ImageFolder`, `Dataset` and some `transform` functions.

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as tt
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader

In [ ]:
sampleDataset = ImageFolder(PARENT_DIR)
"Total Length of Dataset :", len(sampleDataset)

In [ ]:
classes = sampleDataset.classes
classes

In [ ]:
sampleDataset = ImageFolder(PARENT_DIR, transform = tt.ToTensor()) # Changes from pixel to tensor

In [ ]:
for img, _ in sampleDataset:
    print(img.size())
    break

In [ ]:
img, _ = sampleDataset[848]
print(img.size())

In [ ]:
img, _ = sampleDataset[8448]
print(img.size())

Now if you look back, we have images varying from size to size, Let's `resize` every image and add some `agumentation` to the images to avoid `overfitting`

In [ ]:
# Changing to Tensor and Applying Data Agumentation

dataFolder = ImageFolder(
    PARENT_DIR,
    tt.Compose([
        tt.Resize(64), # Resize to 3x64x64
        tt.RandomCrop(32, pad_if_needed = True), # Cropping Half Image
        tt.RandomRotation(degrees = 55),
        tt.ToTensor()
    ])
)

In [ ]:
for i in [4, 44, 444, 4444]:
    img, _ = dataFolder[i]
    print(f"Image Shape: {img.size()}")

In [ ]:
def plotTensor(num):
    """Plot Tensors after Image Augmentation"""
    img, _ = dataFolder[num]
    plt.imshow(img.permute((1, 2, 0)))
    plt.show()

In [ ]:
plotTensor(7654)

In [ ]:
plotTensor(76)

As we see, Our Dataset is at final stage, Just splitting for traning and validation is left, Our Model will not like to overfit but `DataAugmentation` and `Validating`.
Let's split the dataset. We need only `2%` of the original dataset for validation

In [ ]:
valFrac = 0.2
valLength = int(valFrac * len(dataFolder))

trainLength = len(dataFolder) - valLength

trainLength, valLength

In [ ]:
train_ds, val_ds = random_split(dataFolder, [trainLength, valLength])

len(train_ds), len(val_ds)

and Now let's create a `DataLoader` with **128** batches with Pytorch

In [ ]:
batchSize = 128

train_dl = DataLoader(
    train_ds, 
    batch_size = batchSize,
    shuffle = True,
    num_workers = 4,
    pin_memory = True
)

val_dl = DataLoader(
    val_ds, 
    batch_size = batchSize,
    num_workers = 4,
    pin_memory = True
)

We saw as seperate Images, Let's vizualize as batches

In [ ]:
def plotBatch(dataLoader):
    """Function to Plot as Batch"""
    for images, label in dataLoader:
        fig, ax = plt.subplots(figsize = (14, 8))
        ax.set_xticks([]), ax.set_yticks([])
        ax.imshow(make_grid(images, nrow = 16).permute((1, 2, 0)))
        break
    
plotBatch(train_dl)

In [ ]:
plotBatch(val_dl)

### Step - 5 Config GPU

In this step we setup our code that can even run on `GPU/CUDA` if available

In [ ]:
def getDevice():
    # Function to Get Default Device
    if torch.cuda.is_available():
        return torch.device('cuda')
    return torch.device('cpu')

def transferToDevice(data, device):
    """
    Function to transfer Model and DataLoader to GPU
    """
    if isinstance(data, (list, tuple)):
        return [transferToDevice(x, device) for x in data]
    return data.to(device, non_blocking = True)


class DeviceDataLoader():
    def __init__(self, dataLoader, device):
        self.dataLoader = dataLoader
        self.device = device
    
    def __iter__(self):
        for b in self.dataLoader:
            yield transferToDevice(b, self.device)
        
    def __len__(self):
        return len(self.dataLoader)

We created config functions and classes, Let's test it
1. By checking if gpu is available
2. By Changing the images from the data loader

In [ ]:
defaultDevice = getDevice()
defaultDevice

In [ ]:
for images, _ in train_dl:
    print(f"Device Before: {images.device}")
    configedImage = transferToDevice(images, defaultDevice)
    print(f"Device After: {configedImage.device}")
    break

We transfered Sample Tensor from CPU to Cuda. Now let's change the whole DataLoader

In [ ]:
train_dl = DeviceDataLoader(train_dl, defaultDevice)
val_dl = DeviceDataLoader(val_dl, defaultDevice)

In [ ]:
train_dl.device, val_dl.device

### Step - 6 Model Traning

We will use `Convoluation Neural Network`.

In [ ]:
class CNN_Model(nn.Module):
    def __init__(self):
        super().__init__()

        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size = 3, padding = 1, stride = 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size = 3, padding = 1, stride = 1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size = 3, padding = 1, stride = 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, kernel_size = 3, padding = 1, stride = 1),
            nn.Flatten(),
            nn.Linear(256 * 8 * 8, 512),
            nn.ReLU(),
            nn.Linear(512, 3)
        )

    def forward(self, xb):
        out = self.network(xb)
        return out

In [ ]:
for images, labels in train_dl:
    model = CNN_Model().to(defaultDevice)
    print(images.shape)
    out = model(images)
    print(out.shape)
    break

We changed the dimension from `3 Channel` RGB image with 32 * 32 Width and Height, Now we have three output neurons which outputs the three probability.

In [ ]:
def train_step(model, dataloader, loss_fn, optimizer):
    model.train()
    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

In [ ]:
def val_step(model, dataloader, loss_fn):
    model.eval() 
    val_loss, val_acc = 0, 0
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            test_pred_logits = model(X)
            loss = loss_fn(test_pred_logits, y)
            val_loss += loss.item()
            test_pred_labels = test_pred_logits.argmax(dim=1)
            val_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
    val_loss = val_loss / len(dataloader)
    val_acc = val_acc / len(dataloader)
    return val_loss, val_acc

In [ ]:
def train(model,
          train_dataloader,
          test_dataloader,
          optimizer,
          loss_fn = nn.CrossEntropyLoss(),
          epochs = 5):
    results = {"train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": []
    }
    
    for epoch in range(epochs):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        val_loss, val_acc = val_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn)
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"val_loss: {val_loss:.4f} | "
            f"val_acc: {val_acc:.4f}"
        )
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["vall_loss"].append(val_loss)
        results["vall_acc"].append(val_acc)
    return results

In [ ]:
cnnModel = CNN_Model().to(defaultDevice)

In [ ]:
epochs = 5
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=cnnModel.parameters(), lr=0.001)

In [ ]:
cnnModelResults = train(model = cnnModel, 
                        train_dataloader = train_dl,
                        test_dataloader = val_dl,
                        optimizer = optimizer,
                        loss_fn = loss_fn, 
                        epochs = epochs)